# Multi-agent application

In [1]:
%pip install -U langchain-mcp-adapters langgraph "langchain[openai]"

  Attempting uninstall: mcp
    Found existing installation: mcp 1.9.1
    Uninstalling mcp-1.9.1:
      Successfully uninstalled mcp-1.9.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from dotenv import load_dotenv
from langchain_openai import AzureOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import AzureChatOpenAI
import requests
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool

env_path = r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\.env"
load_dotenv(dotenv_path=env_path, override=True)

# Access the environment variables
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
tavily_api_key = os.getenv("TAVILY_API_KEY")


In [5]:
# Initialize the Azure OpenAI model
llm = AzureChatOpenAI(
    openai_api_version=openai_api_version,
    azure_deployment=azure_chat_deployment,
)

# Create a human message and invoke the model
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
response = llm.invoke([message])

# Print the response
print(response)

content="J'aime programmer." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 19, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BiNbfZejeyHZVDQ8WJjfKT7gHgkCW', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'f

In [3]:
import json
import random
from datetime import datetime

# Sample list of stock symbols and sectors
assets = [
    {"symbol": "AAPL", "sector": "Technology"},
    {"symbol": "GOOGL", "sector": "Technology"},
    {"symbol": "MSFT", "sector": "Technology"},
    {"symbol": "AMZN", "sector": "Consumer Discretionary"},
    {"symbol": "TSLA", "sector": "Consumer Discretionary"},
    {"symbol": "JNJ", "sector": "Healthcare"},
    {"symbol": "NVDA", "sector": "Technology"},
    {"symbol": "XOM", "sector": "Energy"},
    {"symbol": "META", "sector": "Communication Services"},
    {"symbol": "V", "sector": "Financials"},
    {"symbol": "PG", "sector": "Consumer Staples"},
    {"symbol": "BABA", "sector": "Consumer Discretionary"},
    {"symbol": "JPM", "sector": "Financials"},
    {"symbol": "KO", "sector": "Consumer Staples"},
    {"symbol": "DIS", "sector": "Communication Services"},
    {"symbol": "PFE", "sector": "Healthcare"},
]

def generate_portfolio(assets):
    portfolio = []
    for asset in assets:
        quantity = random.randint(5, 100)
        purchase_price = round(random.uniform(50, 3000), 2)
        total_invested = round(quantity * purchase_price, 2)
        purchase_date = datetime.strftime(datetime(2022, random.randint(1,12), random.randint(1,28)), "%Y-%m-%d")

        portfolio.append({
            "symbol": asset["symbol"],
            "sector": asset["sector"],
            "quantity": quantity,
            "purchase_price": purchase_price,
            "total_invested": total_invested,
            "purchase_date": purchase_date
        })
    return portfolio

# Generate and save portfolio
portfolio_data = generate_portfolio(assets)

with open("sample_portfolio.json", "w") as f:
    json.dump(portfolio_data, f, indent=4)

print("✅ Portfolio JSON saved to 'sample_portfolio.json'")


✅ Portfolio JSON saved to 'sample_portfolio.json'


# Search Agent

In [1]:
from typing import Annotated, List
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict
from datetime import datetime




USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
tavily_tool = TavilySearchResults(max_results=5, tavily_api_key=tavily_api_key)

search_prompt = f"""
You are an expert financial researhcer. You search financial markets and provide insights based on the latest data. You can search for information online to answer questions about stocks, market trends, and financial news.
You will be provided with a search query, and you should return the most relevant information you find.
You will use the Tavily search tool to find information. If you cannot find relevant information, you should respond with "I could not find relevant information for your query."
You should only return the information you find, and not any additional commentary or analysis.
Make sure to always search up-to-date information based on today's date: {datetime.today().strftime('%Y-%m-%d')}
"""

search_agent = create_react_agent(llm, tools=[tavily_tool], name = "search_agent", prompt = search_prompt)



# Read Portfolio Agent

In [7]:
import json

@tool
def read_sample_portfolio(json_path: str = "sample_portfolio.json") -> str:
    """
    Reads the sample_portfolio.json file and returns its content as a string.
    Each entry includes the stock symbol, sector, quantity, purchase price, and purchase date.
    """
    if not os.path.exists(json_path):
        return f"File not found: {json_path}"

    with open(json_path, "r") as f:
        portfolio = json.load(f)

    if not isinstance(portfolio, list):
        return "Unexpected portfolio format."

    response = "Sample Portfolio:\n"
    for stock in portfolio:
        response += (
            f"- {stock['symbol']} ({stock['sector']}): "
            f"{stock['quantity']} shares @ ${stock['purchase_price']} "
            f"(Bought on {stock['purchase_date']})\n"
        )
    return response

In [11]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from typing_extensions import TypedDict

portfolio_prompt = """
You are a personal finance assistant. You help users understand their investment portfolios and provide insights based on the data provided.
You can read user's portfolio leveraging the read_sample_portfolio tool.
"""

read_portfolio_agent = create_react_agent(llm, tools=[read_sample_portfolio], name = "read_portfolio_agent", prompt = portfolio_prompt)




# Report generator Agent

In [12]:
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional
from typing_extensions import TypedDict

# Define a real directory path
REAL_DIRECTORY = Path(r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\output")

#_TEMP_DIRECTORY = TemporaryDirectory()
WORKING_DIRECTORY = Path(REAL_DIRECTORY)

In [13]:
@tool
def write_document(
    content: Annotated[str, "Text content to be written into the document."],
    file_name: Annotated[str, "File path to save the document."],
) -> Annotated[str, "Path of the saved document file."]:
    """Create and save a text document."""
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Document saved to {file_name}"



In [14]:
report_prompt = """

You are an expert report generator. Given the input from other agents, you generate a detailed report on how to optimize the provided portfolio.
The report will have the following outline:

-------------------------------
**Introduction on market landscape**
**Portfolio Overview**
**Investment Strategy**
**Performance Analysis**
**Recommendations**
**Conclusion**

**References**
--------------------------------

Once generated the report, save it using your write_document tool.


"""

doc_writer_agent = create_react_agent(
    llm,
    tools=[write_document],
    prompt=report_prompt,
    name = "doc_writer_agent",
)



# All together

In [21]:
! pip install langgraph-supervisor

In [ ]:
from langchain_openai import ChatOpenAI

from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent


# Create supervisor workflow
workflow = create_supervisor(
    [search_agent, doc_writer_agent, read_portfolio_agent],
    model=llm,
    prompt=
        "You are a smart team supervisor managing multiple agents. Analyze the user input and delegate to the appropriate agent:\n"
        "- If the input asks for your current financial position, use read_portfolio_agent.\n"
        "- If the input contains 'search' or asks to find something online, use search_agent.\n"
        "- If the input asks to generate an output report, use doc_writer_agent.\n"
        "- For all other questions or queries, use your general knowledge.\n"
        "Choose the most appropriate agent based on the user's input."
)

# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "how can I optimize my portfolio?"
        }
    ]
})

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# Initialize checkpointer
checkpointer = InMemorySaver()
app = workflow.compile(checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}

# Main interaction loop
while True:
    user_input = input("\nEnter your query (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("Goodbye!")
        break

    result = app.invoke({
        "messages": [{
            "role": "user",
            "content": user_input
        }]
    }, config=config)

    for m in result["messages"]:
        print(m.content)

In [15]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def format_step(step):
    message = step.get("messages", [])[-1] if "messages" in step else None
    if isinstance(message, HumanMessage):
        return f"\n👤 **User:** {message.content}"
    
    if isinstance(message, ToolMessage):
        return f"\n🔧 **Tool [{message.name}] says:** {message.content}"
    
    if isinstance(message, AIMessage):
        if "tool_calls" in message.additional_kwargs:
            tool_call = message.additional_kwargs["tool_calls"][0]
            tool_name = tool_call["function"]["name"]
            return f"\n🤖 **Supervisor:** Calling 🛠️ `{tool_name}`..."
        if message.content.strip():
            return f"\n🤖 **Agent [{message.name}] says:** {message.content}"

    return ""


In [16]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent


# Create supervisor workflow
workflow = create_supervisor(
    [search_agent, doc_writer_agent, read_portfolio_agent],
    model=llm,
    prompt=
        "You are a smart team supervisor managing multiple agents. Use all the agents to accomplish the task:\n"
        "- Use read_portfolio_agent to access the user's financial position. Make sure to invoke it with a query that already includes the stocks of user's portfolio so that it can perform the search.\n"
        "- Use search_agent to get real time information on financial data.\n"
        "- Use doc_writer_agent to generate and save a structured report.\n"
)

# Compile and run
app = workflow.compile()
# result = app.invoke({
#     "messages": [
#         {
#             "role": "user",
#             "content": "how can I optimize my portfolio?"
#         }
#     ]
# })



In [18]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def visual_print_messages(chunk):
    """Print messages with emoji and role labels."""
    messages = chunk.get("supervisor", {}).get("messages", [])
    if not messages:
        return

    for msg in messages:
        if isinstance(msg, HumanMessage):
            print(f"\n👤 [User]: {msg.content}")

        elif isinstance(msg, AIMessage):
            name = msg.name or "AI"
            if "tool_calls" in msg.additional_kwargs:
                for call in msg.additional_kwargs["tool_calls"]:
                    tool = call["function"]["name"]
                    print(f"\n🔁 [{name}] → calls 🛠️ `{tool}`")
            elif msg.content.strip():
                print(f"\n🤖 [{name}]: {msg.content.strip()}")

        elif isinstance(msg, ToolMessage):
            print(f"\n📦 [Tool {msg.name}]: {msg.content}")

        else:
            print(f"\n❓ [Unknown]: {msg.content}")

# Run your streaming loop
for chunk in app.stream({
    "messages": [
        {
            "role": "user",
            "content": "how can I optimize my portfolio?"
        }
    ]
}):
    visual_print_messages(chunk)

# Final history (if needed later)
final_history = chunk["supervisor"]["messages"]



👤 [User]: how can I optimize my portfolio?

🔁 [supervisor] → calls 🛠️ `transfer_to_read_portfolio_agent`

📦 [Tool transfer_to_read_portfolio_agent]: Successfully transferred to read_portfolio_agent

👤 [User]: how can I optimize my portfolio?

🔁 [supervisor] → calls 🛠️ `transfer_to_read_portfolio_agent`

📦 [Tool transfer_to_read_portfolio_agent]: Successfully transferred to read_portfolio_agent

🤖 [read_portfolio_agent]: Your portfolio consists of a mix of sectors and diverse stocks. Here are a few steps for optimizing your portfolio:

### 1. **Sector Allocation**
   - Technology dominates your portfolio with stocks such as $AAPL, $GOOGL, $MSFT, and $NVDA.
   - Ensure your portfolio isn't overly reliant on one sector. Consider reallocating or diversifying into underrepresented sectors like Healthcare, Consumer Staples, or Energy if needed.

### 2. **High Volatility Stocks**
   - Stocks like $TSLA, $NVDA, and $DIS are typically more volatile. Assess your risk tolerance levels. If you’re

In [17]:
# Main interaction loop
while True:
    user_input = input("\nEnter your query (or 'exit' to quit): ")

    if user_input.lower() == 'exit':
        print("👋 Goodbye!")
        break

    result = app.invoke({
        "messages": [{"role": "user", "content": user_input}]
    })

    for m in result["messages"]:
        role = m.type  # 'human', 'ai', 'tool'
        name = getattr(m, "name", None)  # agent/tool name

        if role == "human":
            print(f"\n👤 [User]: {m.content}")
        elif role == "ai":
            label = f"[{name}]" if name else ""
            print(f"\n🤖 [AI {label}]: {m.content}")
        elif role == "tool":
            print(f"\n🧰 [Tool {name}]: {m.content}")
        else:
            print(f"\n🔍 [{role.capitalize()}]: {m.content}")



👤 [User]: how can I optimize my portfolio?

🤖 [AI [supervisor]]: 

🧰 [Tool transfer_to_read_portfolio_agent]: Successfully transferred to read_portfolio_agent

🤖 [AI [read_portfolio_agent]]: Your portfolio contains a mix of stocks across different sectors including Technology, Consumer Discretionary, Healthcare, Energy, Communication Services, Financials, and Consumer Staples. A few steps for optimizing your portfolio include:

### 1. **Understand Sector Allocation**
   - Currently, your portfolio is heavily weighted toward **Technology**, with stocks such as AAPL, GOOGL, MSFT, and NVDA. Over-reliance on one sector could expose your portfolio to higher risks if that sector underperforms.
   - Consider diversifying further into underrepresented sectors like **Energy**, **Consumer Staples**, or **Financials**.

### 2. **Rebalance Your Portfolio**
   - Rebalancing ensures your portfolio aligns with your risk tolerance and goals. For instance:
       - If technology stocks have grown sign

# MCP Integration

In [17]:
import tempfile
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

async def make_agent(server_script: str, model, prompt, name: str):
    errlog_path = tempfile.gettempdir() + f"/{name}_mcp_error.log"
    errlog = open(errlog_path, "w")

    server_params = StdioServerParameters(
        command="python",
        args=[server_script],
    )

    async with stdio_client(server_params, errlog=errlog) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await load_mcp_tools(session)
            agent = create_react_agent(
                model=model,
                tools=tools,
                prompt=prompt,
                name=name
            )
            return agent


In [18]:
from langchain_core.messages import SystemMessage

writer_prompt = """

You are an expert report generator. Given the input from other agents, you generate a detailed report on how to optimize the provided portfolio.
The report will have the following outline:

-------------------------------
**Introduction on market landscape**
**Portfolio Overview**
**Investment Strategy**
**Performance Analysis**
**Recommendations**
**Conclusion**

**References**
--------------------------------

Once generated the report, save it using your write_document tool.


"""

static_prompt_reader = SystemMessage(content="You are a portfolio analysis expert.")
static_prompt_searcher = SystemMessage(content="You are a financial market analyst.")
static_prompt_writer = SystemMessage(content=writer_prompt)

read_agent = await make_agent(r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\mcp\portfolio.py", llm, static_prompt_reader, name="read_portfolio")
search_agent = await make_agent(r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\mcp\web.py", llm, static_prompt_searcher, name="search")
doc_agent = await make_agent(r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\mcp\writer.py", llm, static_prompt_writer, name="doc_writer")


In [21]:
from langgraph.graph import StateGraph, START
from typing import Literal, TypedDict, List
from langchain_core.messages import BaseMessage
from langgraph.types import Command

class State(TypedDict):
    messages: List[BaseMessage]

def make_node(agent):
    def node(state: State) -> Command[Literal["supervisor"]]:
        response = agent.invoke(state)
        return Command(
            goto="supervisor",
            update={"messages": response["messages"]}
        )
    return node

def supervisor_node(state: State) -> Command:
    last_msg = state["messages"][-1].content.lower()
    if "portfolio" in last_msg:
        return Command(goto="read_portfolio")
    elif "search" in last_msg or "market" in last_msg:
        return Command(goto="search")
    elif "report" in last_msg or "write" in last_msg:
        return Command(goto="doc_writer")
    else:
        return Command(goto="supervisor")  # default fallback


In [22]:
builder = StateGraph(State)
builder.add_node("supervisor", supervisor_node)
builder.add_node("read_portfolio", make_node(read_agent))
builder.add_node("search", make_node(search_agent))
builder.add_node("doc_writer", make_node(doc_agent))

builder.add_edge(START, "supervisor")
builder.add_edge("read_portfolio", "supervisor")
builder.add_edge("search", "supervisor")
builder.add_edge("doc_writer", "supervisor")

super_graph = builder.compile()


In [ ]:
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [
        HumanMessage(content="Generate a well structured report on how to improve my portfolio given the market landscape in Q4 2025.")
    ]
}

async for output in super_graph.astream(initial_state, config={"recursion_limit": 10}):
    print(output)
    #print(output["messages"][-1].content)
    print("---")



{'supervisor': None}
---
{'read_portfolio': {'messages': [HumanMessage(content='Generate a well structured report on how to improve my portfolio given the market landscape in Q4 2025.', additional_kwargs={}, response_metadata={}, id='4018dc2c-30d0-415e-922d-e6e4584c81eb'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zbeMbMYlhLhSridcyAFVYXX2', 'function': {'arguments': '{}', 'name': 'read_sample_portfolio'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 135, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BgpKWI5DPxRWeZZLyXjq6xeZ2dK78', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results':

CancelledError: 

In [28]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio

# Create a temporary log file to use instead of sys.stderr
errlog_path = tempfile.gettempdir() + "/mcp_error.log"
errlog = open(errlog_path, "w")

# Math Server Parameters
server_params = StdioServerParameters(
    command="python",
    args=[r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\mcp\mcp_server.py"],
    env=None,
    stderr=None
)

async def main():
    async with stdio_client(server_params, errlog) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # List available prompts
            response = await session.list_prompts()
            print("\n/////////////////prompts//////////////////")
            for prompt in response.prompts:
                print(prompt)

            # List available resources
            response = await session.list_resources()
            print("\n/////////////////resources//////////////////")
            for resource in response.resources:
                print(resource)

            # List available resource templates
            response = await session.list_resource_templates()
            print("\n/////////////////resource_templates//////////////////")
            for resource_template in response.resourceTemplates:
                print(resource_template)

            # List available tools
            response = await session.list_tools()
            print("\n/////////////////tools//////////////////")
            for tool in response.tools:
                print(tool)

            # Get a prompt
            prompt = await session.get_prompt("example_prompt", arguments={"question": "what is 2+2"})
            print("\n/////////////////prompt//////////////////")
            print(prompt.messages[0].content.text)

            # Read a resource
            content, mime_type = await session.read_resource("greeting://Alice")
            print("\n/////////////////content//////////////////")
            print(mime_type[1][0].text)

            # Call a tool
            result = await session.call_tool("add", arguments={"a": 2, "b": 2})
            print("\n/////////////////result//////////////////")
            print(result.content[0].text)

await main()


/////////////////prompts//////////////////
name='example_prompt' description='Example prompt description' arguments=[PromptArgument(name='question', description=None, required=True)]
name='system_prompt' description='System prompt description' arguments=[]

/////////////////resources//////////////////
uri=Url('config://app') name='get_config' description='Static configuration data' mimeType='text/plain' size=None annotations=None

/////////////////resource_templates//////////////////
uriTemplate='greeting://{name}' name='get_greeting' description='Get a personalized greeting' mimeType=None annotations=None

/////////////////tools//////////////////
name='add' description='Add two numbers' inputSchema={'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'addArguments', 'type': 'object'} annotations=None
name='multiply' description='Multiply two numbers' inputSchema={'properties': {'a': {'title': 'A', 'type': 'i

In [ ]:
from typing import List
from typing_extensions import TypedDict
from typing import Annotated

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.checkpoint.memory import MemorySaver

from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_mcp_adapters.resources import load_mcp_resources
from langchain_mcp_adapters.prompts import load_mcp_prompt
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

import asyncio

# Math Server Parameters
server_params = StdioServerParameters(
    command="python",
    args=[r"C:\Users\vaalt\OneDrive\Desktop\Projects\Eventi speaker\Oxford AI Summit\code\mcp\mcp_server.py"],
    env=None,
)

async def create_graph(session):
    
    tools = await load_mcp_tools(session)
    llm_with_tool = llm.bind_tools(tools)

    system_prompt = await load_mcp_prompt(session, "system_prompt")
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt[0].content),
        MessagesPlaceholder("messages")
    ])
    chat_llm = prompt_template | llm_with_tool

    # State Management
    class State(TypedDict):
        messages: Annotated[List[AnyMessage], add_messages]

    # Nodes
    def chat_node(state: State) -> State:
        state["messages"] = chat_llm.invoke({"messages": state["messages"]})
        return state

    # Building the graph
    graph_builder = StateGraph(State)
    graph_builder.add_node("chat_node", chat_node)
    graph_builder.add_node("tool_node", ToolNode(tools=tools))
    graph_builder.add_edge(START, "chat_node")
    graph_builder.add_conditional_edges("chat_node", tools_condition, {"tools": "tool_node", "__end__": END})
    graph_builder.add_edge("tool_node", "chat_node")
    graph = graph_builder.compile(checkpointer=MemorySaver())
    return graph

async def main():
    config = {"configurable": {"thread_id": 1234}}
    async with stdio_client(server_params, errlog) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # Check available tools
            tools = await load_mcp_tools(session)
            print("Available tools:", [tool.name for tool in tools])

            # Check available prompts
            prompts = await load_mcp_prompt(session, "example_prompt", arguments={"question": "what is 2+2"})
            print("Available prompts:", [prompt.content for prompt in prompts])
            prompts = await load_mcp_prompt(session, "system_prompt")
            print("Available prompts:", [prompt.content for prompt in prompts])

            # Check available resources
            resources = await load_mcp_resources(session, uris=["greeting://Alice", "config://app"])
            print("Available resources:", [resource.data for resource in resources])

            # Use the MCP Server in the graph
            agent = await create_graph(session)
            while True:
                message = input("User: ")
                response = await agent.ainvoke({"messages": message}, config=config)
                print("AI: "+response["messages"][-1].content)

await main()

Available tools: ['add', 'multiply']
Available prompts: ['\n    You are a math assistant. Answer the question.\n    Question: what is 2+2\n    ']
Available prompts: ['\n    You are an AI assistant use the tools if needed.\n    ']
Available resources: ['Hello, Alice!', 'App configuration here']
AI: Hello! How can I assist you today?
AI: 2 + 4 equals 6.
AI: I currently have access to the following tools:

1. **Add**: This tool adds two numbers together.
2. **Multiply**: This tool multiplies two numbers together.
3. **Multi-tool Use (Parallel)**: This tool allows me to execute multiple tools simultaneously, useful for performing multiple operations independently at the same time.

Let me know how you'd like to use them!
AI: Alright, if you need any assistance in the future, feel free to reach out. Have a great day! 😊


: 